https://excelsior-cjh.tistory.com/167
https://techblog-history-younghunjo1.tistory.com/134#:~:text=%EC%A0%81%EC%A0%88%ED%95%9C%20%EC%A3%BC%EC%84%B1%EB%B6%84%20%EA%B0%9C%EC%88%98%EB%A5%BC%20%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0%20%EC%9C%84%ED%95%B4%EC%84%9C%20%EC%9A%B0%EC%84%A0%EC%9D%80%20%EC%9B%90%EB%B3%B8,%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%EC%9D%84%20%EC%83%9D%EC%84%B1%ED%95%98%EC%9E%90.&text=%EC%9C%84%EC%99%80%20%EA%B0%99%EC%9D%B4%20%EC%A3%BC%EC%84%B1%EB%B6%84%EC%9D%84%208%EA%B0%9C%EB%A1%9C%20%EC%84%A4%EC%A0%95%ED%96%88%EC%9D%84%20%EB%95%8C%20%EA%B2%B0%EA%B3%BC%EA%B0%80%20%EB%82%98%EC%99%94%EB%8B%A4.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn 
import seaborn as sns 
import os 
from tqdm import tqdm 
from glob import glob 
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 로드 

In [ ]:
df = pd.read_csv('./Data/winequality-white.csv')
df['class'] = df['quality']
df = df.drop(columns = ['quality'])

In [ ]:
from sklearn.datasets import fetch_kddcup99
data = fetch_kddcup99()
df = pd.DataFrame(data.data)
df.columns = data['feature_names']
df['class'] = data['target']
df = df.sample(frac=1,random_state=42).reset_index(drop=True)

# 기본 전처리 

In [ ]:
# 결측치 확인 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


## Categorical Data Embedding 
- 카테고리 데이터를 0,1 ... 등으로 변환 함 

In [ ]:
#Label Embedding Dict
target_columns = ['protocol_type','service','flag','class']


def make_label_embedding(column):
    return {value:i for i,value in enumerate(np.unique(df[column]))}

class embeder:
    __init__ = 0 

for column in target_columns:
    exec(f'embeder.{column}_label_embedding = {make_label_embedding(column)}')
    print(f'{column}_label_embedding')

column = target_columns[0]
for column in target_columns:
    exec(f'encoder = embeder.{column}_label_embedding')
    df[column] = df[column].apply(lambda x : encoder[x])    

protocol_type_label_embedding
service_label_embedding
flag_label_embedding
class_label_embedding


## 컬럼 제거 
- 모두 0인 값을 갖는 컬럼 제거 

In [ ]:
target_columns = ['sequence','subject','step']
df = df.drop(columns = target_columns)

In [ ]:
print(np.sum(df.min().values))
#min 값은 모두 0을 가짐; max 값이 0인 feature 찾아서 제거 
target_columns = df.max().reset_index()[df.max().reset_index()[0]==0]['index'].values
df = df.drop(columns = target_columns)

-4115.289070075064


## Train-Valid 분리 

In [ ]:
length = len(df) 
train_length = int(length*0.7)

train_x = np.array(df.drop(columns = ['class']).iloc[:train_length])
valid_x = np.array(df.drop(columns = ['class']).iloc[train_length:])

train_y = np.array(df['class'].iloc[:train_length])
valid_y = np.array(df['class'].iloc[train_length:])


## Scaling 

In [ ]:
class scaling:
    def __init__(self,train_x):
        self.x = train_x  
        self.min, self.max = self.make_scaling_value()
    def make_scaling_value(self):
        min_value = np.min(self.x,axis=0) 
        max_value = np.max(self.x,axis=0) 
        return min_value, max_value 
    def __call__(self,x):
        scaled_value = (x-self.min)/(self.max-self.min)
        return scaled_value 
        
#트레인 데이터를 기준으로 스케일링 진행 
scaler = scaling(train_x)
train_x = scaler(train_x)
valid_x = scaler(valid_x)        

# 차원 축소 

In [ ]:
from sklearn.decomposition import PCA

#PCA를 수행하기 전 적절한 주성분 개수를 파악해야 함 
pca = PCA(n_components=len(df.columns)-1)
pca_array = pca.fit_transform(train_x)
pca_df = pd.DataFrame(pca_array,index=pd.DataFrame(train_x).index,
                      columns = [f"pca{num+1}" for num in range(len(df.columns)-1)])
                      
#개별 고윳값 즉, 각 주성분 마다 고윳값이 0.7이상인 주성분들, 
#누적기여율이 80% 이상이 넘어거지는 지점까지의 주성분들을 기준으로 하여 적절한 주성분 개수를 설정
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(len(df.columns)-1)]))
result['누적기여율'] = result['기여율'].cumsum()
result                      

,설명가능한 분산 비율(고윳값),기여율,누적기여율
pca1,0.050316,0.348686,0.348686
pca2,0.024608,0.170535,0.519221
pca3,0.017589,0.121894,0.641116
pca4,0.015877,0.110030,0.751146
pca5,0.012558,0.087027,0.838172
pca6,0.006362,0.044088,0.882261
pca7,0.004955,0.034341,0.916602
pca8,0.004655,0.032262,0.948864
pca9,0.003976,0.027553,0.976416
pca10,0.003295,0.022833,0.999249


# 모델링 

In [ ]:
from sklearn.linear_model import  RidgeClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
model = RidgeClassifier()
model.fit(train_x,train_y)
y_ = model.predict(valid_x)
print(classification_report(y_true= valid_y,y_pred = y_))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        37
           5       0.58      0.53      0.55       394
           6       0.59      0.73      0.66       733
           7       0.48      0.37      0.42       262
           8       0.00      0.00      0.00        42

    accuracy                           0.57      1470
   macro avg       0.28      0.27      0.27      1470
weighted avg       0.54      0.57      0.55      1470



In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
pca_train_x = pca.fit_transform(train_x.astype(np.float16))
pca_valid_x = pca.transform(valid_x.astype(np.float16))

model = RidgeClassifier()
model.fit(pca_train_x,train_y)
y_ = model.predict(pca_valid_x)
print(classification_report(y_true= valid_y,y_pred = y_))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        37
           5       0.55      0.53      0.54       394
           6       0.60      0.71      0.65       733
           7       0.46      0.38      0.42       262
           8       0.00      0.00      0.00        42

    accuracy                           0.57      1470
   macro avg       0.27      0.27      0.27      1470
weighted avg       0.53      0.57      0.54      1470



In [ ]:
from sklearn.manifold import MDS 
pca = MDS(n_components=7)
pca_train_x = pca.fit_transform(train_x.astype(np.float16))
pca_valid_x = pca.transform(valid_x.astype(np.float16))

model = RidgeClassifier()
model.fit(pca_train_x,train_y)
y_ = model.predict(pca_valid_x)
print(classification_report(y_true= valid_y,y_pred = y_))

AttributeError: 'MDS' object has no attribute 'transform'

In [ ]:
from sklearn.manifold import MDS 
pca = MDS(n_components=7)
pca_train_x = pca.fit_transform(train_x.astype(np.float16))
pca_valid_x = pca.transform(valid_x.astype(np.float16))

model = RidgeClassifier()
model.fit(pca_train_x,train_y)
y_ = model.predict(pca_valid_x)
print(classification_report(y_true= valid_y,y_pred = y_))

AttributeError: 'MDS' object has no attribute 'transform'